In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [11]:
url = "https://www.hooandja.ee/avasta/edukad"

In [12]:
response = requests.get(url)
response

<Response [200]>

In [14]:
data = response.text
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
# Получение названи9й проектов
titles = soup.find_all("div", {"class":"desc"})
for title in titles:
    print(title.text)

In [ ]:
# Получить все ссылки на проекты
jobs = soup.find_all('div', {"class":"image"})
for job in jobs:
    title = job.find('a')
    print (title.get('href'))

In [ ]:
# получение ссылок на Юзеров
links = soup.find_all("a", {"target":"_top"})
for link in links:
     print(link.get('href'))

In [ ]:
# информция по сборам
adresses = soup.find_all("div", {"class":"data"})
for adress in adresses:
    print(adress.text)

### Структурирование данных

In [ ]:
projects = soup.find_all('div', {'class':'project_white_box'})
for project in projects:
    title = project.find('div', {"class":"desc"})
    jobs = project.find('div', {"class":"image"})
    print (title, jobs)

<div class="desc">
<h3>Tartu Tamme Kooli suurlavastus "Prints ja kerjus"</h3>
</div>

In [45]:
print (titles, jobs, links, adresses).text

[<div class="desc">
<h3>Teraapia -ja meelesüsteemideaed Eesti matkapealinna Aegviitu</h3>
</div>, <div class="desc">
<h3>Erik Juhandi debüütluulekogu "Exitus"</h3>
</div>, <div class="desc">
<h3>Lühifilm "Ohver"</h3>
</div>, <div class="desc">
<h3>Mari Kalkuni plaadi "Õunaaia album" ja video väljaandmine</h3>
</div>, <div class="desc">
<h3>Kinkeraamat #õnnelikteekond #becominghappy</h3>
</div>, <div class="desc">
<h3>Lexsoul Dancemachine "Lexplosion II" album</h3>
</div>, <div class="desc">
<h3>Endise spordiajakirjaniku Valeri Maksimovi elulooraamat "Hiiumehe ränd"</h3>
</div>, <div class="desc">
<h3>Narva Airsoft CQB areen</h3>
</div>, <div class="desc">
<h3>VORONJA SKULPTUURIDE AED</h3>
</div>, <div class="desc">
<h3>Film "Ankeet"</h3>
</div>, <div class="desc">
<h3>Curly Strings ja sõbrad - heategevusliku albumi "Lastele" välja andmine</h3>
</div>, <div class="desc">
<h3>"mujal nüüd – elsewhere today" raamatu väljaandmine</h3>
</div>, <div class="desc">
<h3>Investeerimisraamat "Sõlt

AttributeError: 'NoneType' object has no attribute 'text'

In [13]:
tags = soup.find_all('a')
for tag in tags:
    print(tag.get('href'))

/
/avasta/viimati-lisatud
/alusta
/user

/avasta/edukad
/en/avasta/edukad
/ru/avasta/edukad
https://www.hooandja.ee/users/raunoschults
https://www.hooandja.ee/avasta/kategooriad/trukis
https://www.hooandja.ee/projekt/rauno-schultsi-ii-luulekogu-pingil-istusid-kaks-juuti
https://www.hooandja.ee/users/jaan-malin
https://www.hooandja.ee/avasta/kategooriad/trukis
https://www.hooandja.ee/projekt/festivali-%E2%80%9Ehullunud-tartu%E2%80%9C-kogumik
https://www.hooandja.ee/users/helina-victoria-kruup
https://www.hooandja.ee/avasta/kategooriad/trukis
https://www.hooandja.ee/projekt/unustatud-operetip%C3%A4rl-therese-raide-isikuraamat
https://www.hooandja.ee/users/eerikkandler
https://www.hooandja.ee/avasta/kategooriad/muusika
https://www.hooandja.ee/projekt/s%C3%A4deleva-kassi-esimene-album-muusika
https://www.hooandja.ee/users/imre-heero
https://www.hooandja.ee/avasta/kategooriad/trukis
https://www.hooandja.ee/projekt/v%C3%A4rviraamat-haldjaprintsess-ja-tema-s%C3%B5brad
https://www.hooandja.ee/

In [5]:

# Create a dictionary
d = {'key':'value'}
print(d)

{'key': 'value'}


In [6]:
# Update the dictionary
d['new key'] = 'new value'
print(d)

{'key': 'value', 'new key': 'new value'}


In [8]:
# парсинг всех страниц со всех страниц
# ВСтавка(закачка данных со всех страниц) в фаил csv
npo_jobs = {}
job_no = 0
while True:
    
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data,'html.parser')
    jobs = soup.find_all("div", {"class": "project_white_box"})
    
    for job in jobs:
        title = job.find("h3").text
        project_link = job.find("a", {"target": "_top"}).get('href')
        author = job.find("span").text
        author_link = job.find("a", {"target": "_top"}).get('href')
        project = job.find("div", {"class": "image"}).get('href')
        cat = job.find("a", {"target": "_top"}).text
        job_response = requests.get(author_link)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data, 'html.parser')
        job_description = job_soup.find('div', {'class':'w700 leftside'})
        job_attributes_tag = job_soup.find('div', {'class':'user_about'})
        job_attributes = job_attributes_tag.text if job_attributes_tag else "N/A"
        job_no+=1
        npo_jobs[job_no] = [title, author, author_link, project, cat]
        print ('Title:', title, '\nProject_link:', project,
               '\nAuthor:', author, '\nAuthorLink:', author_link,  
               '\nBio:', job_description, 
               '\nCategory:', cat, '\n---')
    url_tag = soup.find('a',{'title':'Go to next page'})
    if url_tag.get('href'):
        url= 'https://www.hooandja.ee' + url_tag.get('href')
        print(url)
    else:
        break

print("Total Jobs:", job_no)
npo_jobs_df = pd.DataFrame.from_dict(npo_jobs, orient = 'index', columns = ['Job Title','Location','Date', 'Link', 'Job Attributes'])
npo_jobs_df.head()
npo_jobs_df.to_csv('edukad.csv')

Title: Seltskonnamäng "Karupong" 
Project_link: None 
Author: ei kysind OÜ  
AuthorLink: https://www.hooandja.ee/users/rainh951 
Bio: <div class="w700 leftside">
<div class="user_about">Oleme viis noort, kes soovivad luua uusi ja huvitavaid seltskonnamänge!</div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: ei kysind OÜ  
---
Title: "Mäepealse lood" (raamat esimesele kooliastmele kuni vanusele 99+) 
Project_link: None 
Author: Maarja Astover  
AuthorLink: https://www.hooandja.ee/users/astovermaarja 
Bio: None 
Category: Maarja Astover  
---
Title: Luuletustega raamat "Sinine värv on otsas" 
Project_link: None 
Author: Marge Pärnits  
AuthorLink: https://www.hooandja.ee/users/maie 
Bio: <div class="w700 leftside">
<div class="user_about">Pildistan, kirjutan või vaatan niisama maailma. Pole paremat asja jõe vaatamisest või akna all istumisest.<br/>
<br/>
On olemas mõned kaantega asjad, mis panevad mind naeratama. N

Title: Lükka hoogu PONIKARIKALE: ÄGEDAD VÕISTLUSLATID JA  MOBIILNE JÄRELKÄRU 
Project_link: None 
Author: Katrin Vellet  
AuthorLink: https://www.hooandja.ee/users/katrinvellet 
Bio: None 
Category: Katrin Vellet  
---
Title: Anna hoogu esimesele kogukonna korvpallitiimile ROCK 
Project_link: None 
Author: Rock korvpalliklubi  
AuthorLink: https://www.hooandja.ee/users/info228 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Rock korvpalliklubi  
---
Title: Lucky Laika kahe aasta kalender 
Project_link: None 
Author: Lucky Laika  
AuthorLink: https://www.hooandja.ee/users/tarieli-lipartia 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Lucky Laika  
---
Title: Eestlane maailmarekordit purustama! 
Project_link: None 
Aut

Title: Praktiline käsiraamat "Veel 100 ideed õpetajale" 
Project_link: None 
Author: Kristi Lõbu  
AuthorLink: https://www.hooandja.ee/users/flicka00 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Kristi Lõbu  
---
Title: Eesti noolemängu meeskond WDF World Cup’ile Rumeenias 
Project_link: None 
Author: Eesti Dartsi koondis 2019  
AuthorLink: https://www.hooandja.ee/users/andrespaal 
Bio: <div class="w700 leftside">
<div class="user_about">Me oleme Rumeenias 2019 WDF World Cupil Eestit esindav dartsi (noolemängu) koondis. Vasakult Erki Selling, Raido Kruusvee, Andres Paal, Meelis Aule. Meie meeskonnas on kõik selle ala kirglikud harrastajad ja meil on soov seda ala Eestis edasi viia ning kanda ka Eesti noolemängu maailmakaardile.</div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category

Title: Uus Tantsukool vajab hoogu!  
Project_link: None 
Author: Alissa Montes  
AuthorLink: https://www.hooandja.ee/users/alyssake 
Bio: <div class="w700 leftside">
<div class="user_about">Tere! Olen Alissa! Elkõige kahele vahvale lapsele ema. Mu elus on alati väga tähtsal kohal olnud muusika ja liikumine. Olen laste ja täiskasvanute õpetamisega tegelenud juba peaaegu 20 aastat.  Praeguseks võin uhkusega öelda, et olen õpetanud erinevaid tantsustiile tuhandetele eri vanuses õpilastele. Süstinud tantsurõõmu paljudesse lastesse ja noortesse. Minu esimeseks ja ainukeseks suureks tantsukoduks on olnud DanceAct Tantsustuudio kus olen 17 imelist aastat  olnud cheerleaders ja  show suuna üks juhtivaid treenereid. <br/>
Kuid aeg on liikuda edasi. On aeg avada omanäoline tantsukool, mis saab olema just selline koht kuhu ma rõõmuga toon ka oma lapsed ja loomulikult avan uksed ja ootan  kõiki  lapsi, noori ja täiskasvanuid nautima hea energiaga ja atmosfääriga trenne.  </div>
<h3 class="dotted" 

Title: Discgolfi pargi rajamine Ebaverre 
Project_link: None 
Author: Väike-Maarja Vallavalitsus  
AuthorLink: https://www.hooandja.ee/users/signesabas 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Väike-Maarja Vallavalitsus  
---
Title: Vabaõhulavastus "Taevas, muld ja tulevik" 
Project_link: None 
Author: Merilin Kirbits  
AuthorLink: https://www.hooandja.ee/users/merilinkirbits 
Bio: None 
Category: Merilin Kirbits  
---
Title: Maria Stuarti uue albumi väljaandmine 
Project_link: None 
Author: Liana Kolodinskaja  
AuthorLink: https://www.hooandja.ee/users/mariastuartband 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Liana Kolodinskaja  
---
Title: Uued ja turvalised masinad Virtsu Kooli tööõpetuseklassi! 
Projec

...

Title: CD „Pranglile“ 
Project_link: None 
Author: tinaraud  
AuthorLink: https://www.hooandja.ee/users/tinaraud 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: tinaraud  
---
Title: VÄRVIME MAAILMA ÄRA! 
Project_link: None 
Author: Annika Klemets; Haavakannu Tall  
AuthorLink: https://www.hooandja.ee/users/annika5 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Annika Klemets; Haavakannu Tall  
---
Title: Ära mind unusta 
Project_link: None 
Author: fil  
AuthorLink: https://www.hooandja.ee/users/fil 
Bio: <div class="w700 leftside">
<div class="user_about">Loominguline, ühiskondlikult aktiivne ja kultuuri armastav optimistliku maailmavaatega isiksus</div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h

Title: Uppsar ja Tsuumi uurivad „Tuljakut“ 
Project_link: None 
Author: Raido Bergstein  
AuthorLink: https://www.hooandja.ee/users/zick 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Raido Bergstein  
---
Title: Lasteraamat "Ats läheb uitama" 
Project_link: None 
Author: Andres Aarik  
AuthorLink: https://www.hooandja.ee/users/andresaarik 
Bio: <div class="w700 leftside">
<div class="user_about">Olen vabakutseline illustraator ja kritseldaja..</div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Andres Aarik  
---
Title: Eesti kurtide filmi ja teatri festival 
Project_link: None 
Author: ValgusNoored  
AuthorLink: https://www.hooandja.ee/users/valgusnoortekas 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevus

Title: Teater NO99 lavastus 
Project_link: None 
Author: Teater NO99  
AuthorLink: https://www.hooandja.ee/users/katlin 
Bio: <div class="w700 leftside">
<div class="user_about">Teater NO99 asutati aastal 2004 ja teatri esimene esietendus oli aastal 2005. Teatril on püsiv 7-liikmeline trupp. Igal hooajal tuuakse välja mitmeid uuslavastusi. Lavastajad Tiit Ojasoo ja Ene-Liis Semper ning näitlejad kasutavad tihti improvisatsiooni, selleks et luua lavastuste stseene ja teksti. Lavastus kujunebki sel moel lavastajate ja näitlejate koostöös. Sellisel meetodil loodud lavastused on teatri repertuaaris olulisel kohal.</div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Teater NO99  
---
Title: Muusikaline tantsumuinasjutt "OmaOma"  
Project_link: None 
Author: Annika Viibus  
AuthorLink: https://www.hooandja.ee/users/annikaviibus 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="di

Title: Müstiline raba 
Project_link: None 
Author: Chintis Lundgren  
AuthorLink: https://www.hooandja.ee/users/chintis-lundgren 
Bio: <div class="w700 leftside">
<div class="user_about">Chintis Lundgren on joonistanud tobedaid animatsioone lindudest peaaegu kolm aastat ja näidatud on neid festivalidel üle terve maailma (Kanada, USA, Jaapan, Iraan, UK, Sloveenia jne.). Mõni neist on saanud lausa auhinna. Lähitulevikus on Chintisel plaanis teostada veel kummalisi filme lindudest ning koos nendega mööda festivale ringi lennata ning oma 84. sünnipäevaks loodab autor välja anda erakordselt põhjaliku kogumiku linnuanimatsioonidest.</div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Chintis Lundgren  
---
Title: Kotid karmi ajalooga 
Project_link: None 
Author: Ivar  
AuthorLink: https://www.hooandja.ee/users/ivararulaid 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:

AttributeError: 'NoneType' object has no attribute 'get'

In [9]:
npo_jobs_df = pd.DataFrame.from_dict(npo_jobs, orient = 'index', columns = ['Job Title','Location','Date', 'Link', 'Job Attributes'])
npo_jobs_df.head()
npo_jobs_df.to_csv('edukad.csv')

Total Jobs: 2


In [206]:
npo_jobs_df = pd.DataFrame.from_dict(npo_jobs, orient = 'index', columns = ['Job Title','Location','Date', 'Link', 'Job Attributes', 'Job Description'])


npo_jobs_df.head()


npo_jobs_df.to_csv('edukad.csv')

NameError: name 'npo_jobs' is not defined

In [148]:
data = response.text

In [149]:
soup = BeautifulSoup(data,'html.parser')

In [150]:
jobs = soup.find_all("div", {"class": "project_white_box"})

In [ ]:
for job in jobs:
    title = job.find("h3").text
    project_link = job.find("a", {"target": "_top"}).get('href')
    author = job.find("span").text
    author_link = job.find("a", {"target": "_top"}).get('href')
    project = job.find("div", {"class": "image"}).get('href')
    cat = job.find("a", {"target": "_top"}).text
    print ('Title:', title, '\nProject_link:', project,
           '\nAuthor:', author, '\nAuthorInfo:', author_link,  
           '\nCategory:', cat, '\n---')

In [ ]:
# получение информации со страницы
for job in jobs:
    title = job.find("h3").text
    project_link = job.find("a", {"target": "_top"}).get('href')
    author = job.find("span").text
    author_link = job.find("a", {"target": "_top"}).get('href')
    project = job.find("div", {"class": "image"}).get('href')
    cat = job.find("a", {"target": "_top"}).text
    job_response = requests.get(author_link)
    job_data = job_response.text
    job_soup = BeautifulSoup(job_data, 'html.parser')
    job_description = job_soup.find('div',{'class':'user_about'}).text
    job_attributes_tag = job_soup.find('div',{'class':'w700 leftside'})
    job_attributes = job_attributes_tag.text if job_attributes_tag else "N/A"
    
    print ('Title:', title, '\nProject_link:', project,
           '\nAuthor:', author, '\nAuthorLink:', author_link,  
           '\nBio:', job_description, 
           '\nCategory:', cat, '\n---')

In [196]:
# парсинг всех страниц со всех страниц
# НЕ МЕНЯТЬ КОД!!!
job_no = 0
while True:
    
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data,'html.parser')
    jobs = soup.find_all("div", {"class": "project_white_box"})
    
    for job in jobs:
        title = job.find("h3").text
        project_link = job.find("a", {"target": "_top"}).get('href')
        author = job.find("span").text
        author_link = job.find("a", {"target": "_top"}).get('href')
        project = job.find("div", {"class": "image"}).get('href')
        cat = job.find("a", {"target": "_top"}).text
        job_response = requests.get(author_link)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data, 'html.parser')
        job_description = job_soup.find('div', {'class':'w700 leftside'})
        job_attributes_tag = job_soup.find('div', {'class':'user_about'})
        job_attributes = job_attributes_tag.text if job_attributes_tag else "N/A"
        job_no+=1
        print ('Title:', title, '\nProject_link:', project,
               '\nAuthor:', author, '\nAuthorLink:', author_link,  
               '\nBio:', job_description, 
               '\nCategory:', cat, '\n---')
    url_tag = soup.find('a',{'title':'Go to next page'})
    if url_tag.get('href'):
        url= 'https://www.hooandja.ee' + url_tag.get('href')
        print(url)
    else:
        break
        
print("Total Jobs:", job_no)


Title: Rauno Schultsi II luulekogu "Pingil istusid kaks juuti" 
Project_link: None 
Author: Rauno Schults  
AuthorLink: https://www.hooandja.ee/users/raunoschults 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Rauno Schults  
---
Title: Festivali „Hullunud Tartu“ kogumik 
Project_link: None 
Author: Jaan Malin  
AuthorLink: https://www.hooandja.ee/users/jaan-malin 
Bio: None 
Category: Jaan Malin  
---
Title: Unustatud operetipärl  Therese Raide isikuraamat 
Project_link: None 
Author: helinavictoria  
AuthorLink: https://www.hooandja.ee/users/helina-victoria-kruup 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: helinavictoria  
---
Title: Sädeleva Kassi esimene album "MUUSIKA" 
Project_link: None 
Author: Eerik Kändl

KeyboardInterrupt: 

In [197]:
# парсинг всех страниц со всех страниц
# КОПИЙА

job_no = 0
while True:
    
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data,'html.parser')
    jobs = soup.find_all("div", {"class": "project_white_box"})
    
    for job in jobs:
        title = job.find("h3").text
        project_link = job.find("a", {"target": "_top"}).get('href')
        author = job.find("span").text
        author_link = job.find("a", {"target": "_top"}).get('href')
        project = job.find("div", {"class": "image"}).get('href')
        cat = job.find("a", {"target": "_top"}).text
        job_response = requests.get(author_link)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data, 'html.parser')
        job_description = job_soup.find('div', {'class':'w700 leftside'})
        job_attributes_tag = job_soup.find('div', {'class':'user_about'})
        job_attributes = job_attributes_tag.text if job_attributes_tag else "N/A"
        job_no+=1
        print ('Title:', title, '\nProject_link:', project,
               '\nAuthor:', author, '\nAuthorLink:', author_link,  
               '\nBio:', job_description, 
               '\nCategory:', cat, '\n---')
    url_tag = soup.find('a',{'title':'Go to next page'})
    if url_tag.get('href'):
        url= 'https://www.hooandja.ee' + url_tag.get('href')
        print(url)
    else:
        break
        
print("Total Jobs:", job_no)



Title: Rauno Schultsi II luulekogu "Pingil istusid kaks juuti" 
Project_link: None 
Author: Rauno Schults  
AuthorLink: https://www.hooandja.ee/users/raunoschults 
Bio: <div class="w700 leftside">
<div class="user_about"></div>
<h3 class="dotted" style="display:none;">Viimased tegevused</h3>
<ul class="image_title_list">
</ul>
</div> 
Category: Rauno Schults  
---
Title: Festivali „Hullunud Tartu“ kogumik 
Project_link: None 
Author: Jaan Malin  
AuthorLink: https://www.hooandja.ee/users/jaan-malin 
Bio: None 
Category: Jaan Malin  
---


KeyboardInterrupt: 

In [ ]:
    url_tag = soup.find('a',{'title':'Go to next page'})
        if url_tag.get('href'):
            url= 'https://www.hooandja.ee' + url_tag.get('href')
            print(url)
        else:
            break

    print("Total Jobs:", job_no)
    print ('Title:', title, '\nProject_link:', project,
                   '\nAuthor:', author, '\nAuthorLink:', author_link,  
                   '\nBio:', job_description, 
                   '\nCategory:', cat, '\n---')

In [161]:
print("Total Jobs:", job_no)

Total Jobs: 0


In [162]:
print ('Title:', title, '\nProject_link:', project,
               '\nAuthor:', author, '\nAuthorLink:', author_link,  
               '\nBio:', job_description, 
               '\nCategory:', cat, '\n---')

Title: Lexsoul Dancemachine "Lexplosion II" album 
Project_link: None 
Author: Lexsoul Dancemachine  
AuthorLink: https://www.hooandja.ee/users/353 
Bio: Võib vist öelda, et mu elu on kirju nagu ka mu kirjutiste žanrivalik.

Ca 14 aastat töötasin ajakirjanikuna, lisaks olen viimase 20 aasta jooksul kirjutanud luulet ja laulusõnu, kõnesid ja tervitusi, ürituste stsenaariumeid ja vahetekste, turundusjuhina kodulehekülje ja sotsiaalmeedia sisu. Olen tekste tõlkinud ja toimetanud.

Kas see näitab mitmekülgsust või pealiskaudsust, otsusta ise. Samas - paljud kokad on suurepärased kondiitrid ja vastupidi.

Mu töid on kogumikes avaldatud nii Eestis kui ka Šotimaal.  
Category: Lexsoul Dancemachine  
---


In [ ]:
# не верно
for job in jobs:
    title = job.find("h3").text
    project_link = job.find("a", {"target": "_top"}).get('href')
    author = job.find("span").text
    author_link = job.find("a", {"target": "_top"}).get('href')
    project = job.find("div", {"class": "image"}).get('href')
    cat = job.find("a", {"target": "_top"}).text
    job_response = requests.get(author_link)
    job_data = job_response.text
    job_soup = BeautifulSoup(job_data, 'html.parser')
    # неверная последовательность !
    job_description = job_soup.find('div',{'class':'w700 leftside'}).text
    job_attributes_tag = job_soup.find('div',{'class':'user_about'})
    job_attributes = job_attributes_tag.text if job_attributes_tag else "N/A"
    
    print ('Title:', title, '\nProject_link:', project,
           '\nAuthor:', author, '\nAuthorInfo:', author_link,  
           '\nBio', job_description, 
           '\nCategory:', cat, '\n---')

In [75]:
jobs = soup.find_all("div", {"class": "desc"})
for job in jobs:
    title = job.find("h3").text
    print ('Title: ', title, 'Link: ', block)

Title:  Teraapia -ja meelesüsteemideaed Eesti matkapealinna Aegviitu Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus
Title:  Erik Juhandi debüütluulekogu "Exitus" Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus
Title:  Lühifilm "Ohver" Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus
Title:  Mari Kalkuni plaadi "Õunaaia album" ja video väljaandmine Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus
Title:  Kinkeraamat #õnnelikteekond #becominghappy Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus
Title:  Lexsoul Dancemachine "Lexplosion II" album Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus
Title:  Endise spordiajakirjaniku Valeri Maksimovi elulooraamat "Hiiumehe ränd" Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus
Title:  Narva Airsoft CQB areen Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus
Title:  VORONJA SKULPTUURIDE AED Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus
Title:  Film "Ankeet" Link:  https://www.ho

Title:  Tartu Tamme Kooli suurlavastus "Prints ja kerjus" Link:  https://www.hooandja.ee/projekt/prints-ja-kerjus


In [51]:
print(title.text)

Tartu Tamme Kooli suurlavastus "Prints ja kerjus"


In [50]:
titles = soup.find_all('h3')
for title in titles:
    print(title.text)

Teraapia -ja meelesüsteemideaed Eesti matkapealinna Aegviitu
Erik Juhandi debüütluulekogu "Exitus"
Lühifilm "Ohver"
Mari Kalkuni plaadi "Õunaaia album" ja video väljaandmine
Kinkeraamat #õnnelikteekond #becominghappy
Lexsoul Dancemachine "Lexplosion II" album
Endise spordiajakirjaniku Valeri Maksimovi elulooraamat "Hiiumehe ränd"
Narva Airsoft CQB areen
VORONJA SKULPTUURIDE AED
Film "Ankeet"
Curly Strings ja sõbrad - heategevusliku albumi "Lastele" välja andmine
"mujal nüüd – elsewhere today" raamatu väljaandmine
Investeerimisraamat "Sõltumatu naine. 5 sammu rahalise vabaduseni"
Lasteraamat "Ei taha! Ei taha!"
Puhja kiriku lühtrite restaureerimine
Ruudu Rahumaru fotomaali näitus "Templihoidjad"
Palavikulilled
Joel Remmel "Live at Taff Club" vinüülplaadi väljaandmine
Raamatu "Kõige magusam vili" II osa väljaandmine
Dagö "Küte"
Ansambli Boamadu lühialbumi avaldamine
Madalseiklusrada Laulasmaa Koolile
Culpas - "Leo Põld" Muusikavideo
Kristiina Ehini uue luulekogu "Janu on kõikidel üks" vä

Teraapia -ja meelesüsteemideaed Eesti matkapealinna Aegviitu
Erik Juhandi debüütluulekogu "Exitus"
Lühifilm "Ohver"
Mari Kalkuni plaadi "Õunaaia album" ja video väljaandmine
Kinkeraamat #õnnelikteekond #becominghappy
Lexsoul Dancemachine "Lexplosion II" album
Endise spordiajakirjaniku Valeri Maksimovi elulooraamat "Hiiumehe ränd"
Narva Airsoft CQB areen
VORONJA SKULPTUURIDE AED
Film "Ankeet"
Curly Strings ja sõbrad - heategevusliku albumi "Lastele" välja andmine
"mujal nüüd – elsewhere today" raamatu väljaandmine
Investeerimisraamat "Sõltumatu naine. 5 sammu rahalise vabaduseni"
Lasteraamat "Ei taha! Ei taha!"
Puhja kiriku lühtrite restaureerimine
Ruudu Rahumaru fotomaali näitus "Templihoidjad"
Palavikulilled
Joel Remmel "Live at Taff Club" vinüülplaadi väljaandmine
Raamatu "Kõige magusam vili" II osa väljaandmine
Dagö "Küte"
Ansambli Boamadu lühialbumi avaldamine
Madalseiklusrada Laulasmaa Koolile
Culpas - "Leo Põld" Muusikavideo
Kristiina Ehini uue luulekogu "Janu on kõikidel üks" vä

In [ ]:
title

In [97]:
links = html.find_all("a")

In [98]:
print (links)

[<a href="/" id="logo"><img alt="Home" src="https://www.hooandja.ee/sites/all/themes/hooandjatheme/logo.png"/></a>, <a href="/avasta/viimati-lisatud"><span>Avasta</span>suurepäraseid projekte</a>, <a href="/alusta"><span>Alusta</span> oma projektiga</a>, <a class="primary" href="/user">Logi sisse</a>, <a class="primary" href="">Eesti</a>, <a class="language-link active" href="/avasta/edukad" xml:lang="et">Eesti</a>, <a class="language-link" href="/en/avasta/edukad" xml:lang="en">English</a>, <a class="language-link" href="/ru/avasta/edukad" xml:lang="ru">Русский</a>, <a href="https://www.hooandja.ee/users/info220" target="_top"><span>Eesti Loodus- ja Loomateraapiakeskus </span></a>, <a href="https://www.hooandja.ee/avasta/kategooriad/keskkond" target="_top">Keskkond</a>, <a href="https://www.hooandja.ee/projekt/teraapia-ja-meelesusteemideaed-aegviitu" target="_top"><img src="https://www.hooandja.ee/sites/default/files/styles/200x229/public/project_images/foto_1.jpg?itok=t7YMPrFk" width

In [114]:
  def find_links (links, keyword):
    title = job.find("h3", {"class": "desc"})  
    links_filtered = {}
    for link in links:
        if link.has_attr("href") and link["href"].find(keyword) > -1:
            links_filtered[link["href"]] = 1
    return list(links_filtered.keys())
proj = []
proj.extend(find_links(soup.find_all("a"), "/projekt/"))
pages = []
pages.extend(find_links(soup.find_all("a"), "/avasta/edukad?page"))    

for page in pages:
    r = requests.get("https://www.hooandja.ee" + page)
    html = BeautifulSoup(r.content)
    proj.extend(find_links(html.find_all("a"), "/projekt/"))
with open("hooandja_edukad.txt", "w") as f:
    f.write("\n".join(proj))

In [ ]:
# мультипроцессинг

